In [1]:
from pathlib import Path
from lightly.api import ApiWorkflowClient
from ultralytics import YOLO
import os

Exportar arquivos do Lightly para cada ciclo

In [2]:
LIGHTLY_TOKEN = "6ef4b5e20f6a1dba87a72a9eb4ddceb3f9529cd3d46b94a8" 
DATASET_NAME = "Teste_proportion2"
DATASET_PATH = 'FOCAL/yolov5_format/images'

In [3]:
# Create the LightlyOne client to connect to the API.
client = ApiWorkflowClient(token=LIGHTLY_TOKEN)
client.set_dataset_id_by_name(dataset_name=DATASET_NAME)

# Get all the tags for this dataset
tags = client.get_all_tags()

# Loop over tags and export the filenames
# Note: first tag in `tags` is the newest one 
#       and the last one is always `initial-tag`
tag_files = []
for tag in tags:
    print(tag.name)
    image_names = client.export_filenames_by_tag_name(
        tag_name=tag.name  # name of the tag in the dataset
    )
    file_name = f"{tag.name}.txt"
    file_path = DATASET_PATH + "/" + file_name
    
    with open(file_path, "w") as f:
        f.write(image_names)
    tag_files.append(file_name)
    

ciclo5
ciclo4
ciclo3
ciclo2
ciclo1
initial-tag


In [34]:
def lightly_init(dataset_name, token=LIGHTLY_TOKEN):
    """
    Inicializa o cliente lightl, define o dataset a ser utilizado
    
    Args: 
        token (str): Token de autenticação do cliente Lightly,
        dataset_name (str): Nome do dataset
    """
    #iniciar o cliente e setar dataset
    client = ApiWorkflowClient(token=LIGHTLY_TOKEN)
    client.set_dataset_id_by_name(dataset_name=dataset_name)

    # Pegando todas as tags
    tags = client.get_all_tags()

    # Ordenando na ordem correta de ciclos e tirando o initial-tag
    tags.reverse()
    tags.pop(0)

    '''
    Para cada tag, criar o arquivo de texto acumulado com os nomes no DATASET_PATH,
    pois necessita para o arquivo de configuração do YOLO.    
    '''
    tag_files = []
    image_names = ''
    for tag in tags:
        print(tag.name)
        if image_names == '':
            image_names = client.export_filenames_by_tag_name(tag_name=tag.name)
        else:
            # Adiciona uma nova linha antes de adicionar os nomes da próxima tag
            image_names += '\n' + client.export_filenames_by_tag_name(tag_name=tag.name)

        file_path = DATASET_PATH + "/" + f"{tag.name}.txt"
        
        with open(file_path, "w") as f:
            f.write(image_names)
        tag_files.append(file_path)

    print(f"Arquivos de texto criados! =D")
    

In [35]:
lightly_init(DATASET_NAME, LIGHTLY_TOKEN)

ciclo1
ciclo2
ciclo3
ciclo4
ciclo5
Arquivos de texto criados! =D


Inverter a lista de arquivos

In [4]:
tag_files.reverse()
tag_files

['initial-tag.txt',
 'ciclo1.txt',
 'ciclo2.txt',
 'ciclo3.txt',
 'ciclo4.txt',
 'ciclo5.txt']

In [5]:
tag_files.pop(0)  # remove the initial tag file
print(f"Tag files: {tag_files}")

Tag files: ['ciclo1.txt', 'ciclo2.txt', 'ciclo3.txt', 'ciclo4.txt', 'ciclo5.txt']


#### Com base nos fileNames, para cada iteração vamos
1. Treinar o modelo com base nos filenames de cada ciclo
2. Salvar o histórico de treinamento
3. Salvar as métricas de avaliação do modelo
4. Salvar o modelo treinado

In [6]:
def prepare_yolo_dataset(cycle_name: str, file: str, dataset_path: str):
    
    output_dir = Path(f"config/{cycle_name}")
    os.makedirs(output_dir, exist_ok=True)    
    # img_out = output_dir / "images" / "train"
    # lbl_out = output_dir / "labels" / "train"
    # img_out.mkdir(parents=True, exist_ok=True)
    # lbl_out.mkdir(parents=True, exist_ok=True)
    
    # for path in frame_paths:
    #     img_path = Path(dataset_path) / path
    #     lbl_path = Path(labels_path) / path.with_suffix(".txt").name

    #     shutil.copy(img_path, img_out / img_path.name)
    #     shutil.copy(lbl_path, lbl_out / lbl_path.name)

    # Criando o data.yaml
    yaml = f"""
path: {dataset_path}
train: {file}
val: val 
test: test
nc: 80
names:
  0: person
  1: bicycle
  2: car
  3: motorcycle
  4: airplane
  5: bus
  6: train
  7: truck
  8: boat
  9: traffic light
  10: fire hydrant
  11: stop sign
  12: parking meter
  13: bench
  14: bird
  15: cat
  16: dog
  17: horse
  18: sheep
  19: cow
  20: elephant
  21: bear
  22: zebra
  23: giraffe
  24: backpack
  25: umbrella
  26: handbag
  27: tie
  28: suitcase
  29: frisbee
  30: skis
  31: snowboard
  32: sports ball
  33: kite
  34: baseball bat
  35: baseball glove
  36: skateboard
  37: surfboard
  38: tennis racket
  39: bottle
  40: wine glass
  41: cup
  42: fork
  43: knife
  44: spoon
  45: bowl
  46: banana
  47: apple
  48: sandwich
  49: orange
  50: broccoli
  51: carrot
  52: hot dog
  53: pizza
  54: donut
  55: cake
  56: chair
  57: couch
  58: potted plant
  59: bed
  60: dining table
  61: toilet
  62: tv
  63: laptop
  64: mouse
  65: remote
  66: keyboard
  67: cell phone
  68: microwave
  69: oven
  70: toaster
  71: sink
  72: refrigerator
  73: book
  74: clock
  75: vase
  76: scissors
  77: teddy bear
  78: hair drier
  79: toothbrush
""".strip()

    with open(output_dir / "data.yaml", "w") as f:
        f.write(yaml)

    return output_dir

In [36]:
def train_yolo(cycle_name, data_yaml_path, epochs=20):
    
    #Carregar modelo
    model = YOLO('yolo11n.pt')

    #Treinar modelo
    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        name=f"{cycle_name}",
        project="lightly_selection2",
        # device=[0, 1]
    )

    return results


In [37]:
total_frames = []
for file in tag_files:
    with open(DATASET_PATH + '/' + file, 'r') as f:
        frames = [linha.strip() for linha in f if linha.strip()]
        print(f'qtd de linhas: {len(frames)}')
        total_frames.extend(frames)

    cycle_name = file.split('.txt')[0]
    print(f"\n🚀 Preparando ciclo {cycle_name} com {len(frames)} imagens")

    config_path = prepare_yolo_dataset(cycle_name, file, DATASET_PATH)

    train_txt_path = config_path / f"{cycle_name}_acumulado.txt"
    with open(train_txt_path, 'w') as f:
        f.write('\n'.join(total_frames))


    results = train_yolo(cycle_name, data_yaml_path=str(config_path / "data.yaml"), epochs=1)


qtd de linhas: 34

🚀 Preparando ciclo ciclo1 com 34 imagens
New https://pypi.org/project/ultralytics/8.3.168 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.167 🚀 Python-3.10.12 torch-2.7.0+cu126 CPU (13th Gen Intel Core(TM) i7-1355U)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=config/ciclo1/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale

train: Scanning dress... 0 images, 0 backgrounds, 34 corrupt: 100%|██████████| 34/34 [00:00<00:00, 73131.45it/s]

train: dress/35f5fecb-755a-45d6-ac3d-3feb8299dc87.jpg: ignoring corrupt image/label: [Errno 2] No such file or directory: 'dress/35f5fecb-755a-45d6-ac3d-3feb8299dc87.jpg'
train: dress/7a80fe51-8dfd-4e62-a65a-b192365731fa.jpg: ignoring corrupt image/label: [Errno 2] No such file or directory: 'dress/7a80fe51-8dfd-4e62-a65a-b192365731fa.jpg'
train: dress/be545b93-c60d-4be6-ae3a-64efd37b7fee.jpg: ignoring corrupt image/label: [Errno 2] No such file or directory: 'dress/be545b93-c60d-4be6-ae3a-64efd37b7fee.jpg'
train: hat/7c28df10-5bfc-4deb-b57f-6c292f07b98b.jpg: ignoring corrupt image/label: [Errno 2] No such file or directory: 'hat/7c28df10-5bfc-4deb-b57f-6c292f07b98b.jpg'
train: hat/d7c07f9f-8362-4abc-92aa-193221a7db5b.jpg: ignoring corrupt image/label: [Errno 2] No such file or directory: 'hat/d7c07f9f-8362-4abc-92aa-193221a7db5b.jpg'
train: longsleeve/0354015d-9ed3-4175-9f85-dd5725dce0e1.jpg: ignoring corrupt image/label: [Errno 2] No such file or directory: 'longsleeve/0354015d-9ed3-

RuntimeError: No valid images found in dress.cache. Images with incorrectly formatted labels are ignored. See https://docs.ultralytics.com/datasets for dataset formatting guidance.

In [71]:
config_path.name

'ciclo1'

In [74]:
str(config_path / "data.yaml")

'config/ciclo1/data.yaml'

In [75]:
!ls

FOCAL		  config	     requirements.txt	    utils
LICENSE		  lightly	     schedule_runs_1.ipynb  venv
README.md	  lightly-api	     schedule_runs_5.ipynb  yolo11n.pt
antigos		  lightly_selection  src
clothing_dataset  predictions.zip    train_yolo.ipynb
